Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# Homework 3: Matrix Transforms

In this homework, we will look at different ways to represent and transform matrices.

### Householder similarity transforms

Suppose we are given a square symmetric matrix $A$.  What sort of algorithm could we use to construct a symmetric tridiagonal matrix $\hat A$ such that
$$ A = Q \hat A Q^T $$
where $Q$ is an orthogonal matrix.
We'll construct such a matrix $Q$ out of elementary reflectors, much like we did in Householder QR.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import block_diag
plt.style.use('seaborn')
np.set_printoptions(precision=4)

A = np.random.rand(4, 4)
A = A + A.T # Random symmetric matrix
A

We'll create a Householder-style reflector matrix for column 0, to introduce zeros below the *tridiagonal*.

In [ ]:
def reflector(v):
    return np.eye(len(v)) - 2*np.outer(v, v)

v = A[1:,0].copy()
v[0] -= np.linalg.norm(v)
v = v / np.linalg.norm(v)
F = reflector(v)
Q_0 = block_diag(np.eye(1), F)
Q_0

In [ ]:
# Introduce zeros below the tridiagonal in column 0.
Q_0 @ A

In [ ]:
# Same in the first row
A_1 = Q_0 @ A @ Q_0
A_1

## Efficiency

In the above, we have computed
$$ A_1 := Q_0 A Q_0 $$
(note that $Q_0 = Q_0^T$) using a very inefficient algorithm.
Instead of creating the dense $n\times n$ reflector $F$ above and then multiplying it at $O(n^3)$ cost, write a function that applies the reflector to a matrix at $O(n^2)$ cost.  Recall that
$$ (I - 2 v v^T) B = B - 2 v (v^T B) $$
and also that
$$ B (I - 2 v v^T) = B - 2 (B v) v^T . $$
Also note that you can take the submatrix all but row 0 using `B[1:]` and all but column 0 using `B[:,1:]`.

In [ ]:
def symreflect(A, v):
    """Efficiently compute Q_0 A Q_0 where Q_0 is defined in terms of v.
    """
    B = A.copy() # Make a copy so we don't modify A
    # Modify B by applying reflections.
    # This line applies the reflection on the left, storing the result in-place.
    B[1:] -= 2 * np.outer(v, v @ B[1:])
    # Now you write code to apply a similar reflection on the right.
    # YOUR CODE HERE
    raise NotImplementedError()
    return B

symreflect(A, v)

In [ ]:
def symreflect_naive(A, v):
    F = reflector(v)
    Q_0 = block_diag(np.eye(1), F)
    return Q_0 @ A @ Q_0

assert np.allclose(symreflect(A, v), symreflect_naive(A, v))
print('Tests pass')

## Tridiagonalization

Suppose we implement a sequence of transformations $Q_0, Q_1, \dotsc$ to introduce zeros in successive columns and rows.  Then we'll have
\begin{align}
\hat A &= \dotsb Q_2 \Big( Q_1 ( Q_0 A Q_0 ) Q_1 \Big) Q_2 \dotsb \\
&= \underbrace{\dotsb Q_2 Q_1 Q_0}_{Q^T} A \underbrace{Q_0 Q_1 Q_2 \dotsb}_{Q}
\end{align}
which is an orthogonal reduction to symmetric tridiagonal form.

First, we note that the natural way to store $Q$ is as a list of reflector vectors $v$.
To check our results, we'd like to be able to make $Q$ as an explicit matrix.  This function is a near copy of one in the Linear Algebra notebook.

In [ ]:
def Q_times_x(V, x):
    """Apply orthogonal matrix Q represented as list of Householder reflectors"""
    y = x.copy()
    for v in reversed(V):
        i = -len(v)
        y[i:] -= 2 * v * (v @ y[i:])
    return y

def Q_as_explicit(V):
    m = len(V[0]) + 1
    Q = np.eye(m)
    for i, col in enumerate(Q.T):
        Q[:,i] = Q_times_x(V, col)
    return Q

In [ ]:
def symtridiagonalize(A):
    """Reduce the symmetric matrix A to tridiagonal form using
    orthogonal transformation Q^T A Q.
    """
    B = A.copy()
    n = len(B)
    V = []
    for i in range(n - 2):
        v = B[i+1:,i].copy()
        # Turn v into a normalized vector representing
        # the reflection I - 2 v v^T
        # YOUR CODE HERE
        raise NotImplementedError()
        B[i:, i:] = symreflect(B[i:, i:], v)
        V.append(v)
    return B, Q_as_explicit(V)

Ahat, Q = symtridiagonalize(A)
print(Ahat) # Symmetric and tridiagonal
print('Error:', np.linalg.norm(Q.T @ A @ Q - Ahat))

In [ ]:
Ahat, Q = symtridiagonalize(A)
np.allclose(A, Q @ Ahat @ Q.T)

## Eigensolvers

This reduction to tridiagonal (or, for non-symmetric matrices, Hessenberg) form is the first step for all modern eigensolvers. Eigensolver algorithms such as the [QR Algorithm for eigenvalues](https://en.wikipedia.org/wiki/QR_algorithm) (not to be confused with QR factorization) are necessarily iterative and those iterations are much less expensive when working with tridiagonal or Hessenberg matrices.

## Low-rank matrix compression

Consider the gravitational force from a star at position $x_1$ acting on a star at position $x_0$,
$$ F_{0,1} = G \frac{m_0 m_1}{\lVert x_1 - x_0 \rVert^3} (x_1 - x_0) $$
where $m_0$ and $m_1$ are the masses of each star respectively.
Suppose we have two galaxies of size $n_0 = 100$ and $n_1 = 200$, each randomly distributed around their respective centers.

In [ ]:
def galaxy(center, sigma, n):
    center = np.array(center)
    return center + sigma*np.random.randn(n, 3)

G0 = galaxy([0,0,0], 1, 100)
G1 = galaxy([20,0,0], 1, 200)

plt.plot(G0[:,0], G0[:,1], 'o')
plt.plot(G1[:,0], G1[:,1], 's')
plt.axis('equal');

We can create the dense matrix with the force from galaxy $G_1$ acting on each star in $G_0$.

In [ ]:
def gravity(g0, g1):
    m = g0.shape[0]
    n = g1.shape[0]
    A = np.zeros((3*m, n))
    for i in range(m):
        r = g1.T - np.outer(g0[i,:], np.ones(n))
        A[3*i:3*i+3, :] = r / np.linalg.norm(r, axis=0)**3
    return A

A = gravity(G0, G1)

This matrix can be compressed using an SVD,
$$ U \Sigma V = A $$
where $U$ and $V^T$ have orthonormal columns and $\Sigma$ is diagonal.

In [ ]:
U, S, V = np.linalg.svd(A, full_matrices=False)
plt.semilogy(S)
S[:20]

Create a rank `k` truncated SVD
$$ \hat U \hat S \hat V \approx A $$
that approximates $A$ to a tolerance of less than $10^{-5}$.  This approximation can be applied in $O(mk + nk)$.

In [ ]:
def truncate_approximation(A):
    U, S, V = np.linalg.svd(A, full_matrices=False)
    # YOUR CODE HERE
    raise NotImplementedError()
    return Uhat, Shat, Vhat



In [ ]:
Uhat, Shat, Vhat = truncate_approximation(A)
assert np.linalg.norm(Uhat @ (Shat[:,None] * Vhat) - A) < 1e-5
assert len(Shat) < 20, "Your approximation is too expensive"
print('Tests pass')

### Experiment

Experiment with the distance between the galaxy centers and their size `sigma`.

In [ ]:
dist = 20
sigma = 1

A = gravity(galaxy([0,0,0], sigma, 100),
            galaxy([0,0,dist], sigma, 200))
U, S, V = np.linalg.svd(A, full_matrices=False)
plt.semilogy(S);

Explain the effect of `dist` and `sigma` on the rank `k` of the approximation necessary to attain a given tolerance.  Write your answer below.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()